In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [40]:
import matplotlib.pyplot as plt
import pandas as pd
import re
import string

from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [2]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv", encoding = 'latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)

print(data.shape)

data.fillna("", inplace = True)

display(data)

X = data['text']
y = data['label']

(1000, 2)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
995,So what's the latest? It sounds contradictory ...,0
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1
997,Barb I will call to explain. Are you back in t...,0
998,Yang on travelNot free tonite.May work tomorrow,0


### Let's divide the training and test set into two partitions

In [3]:
# Your code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (800,) (800,)
Test set: (200,) (200,)


## Data Preprocessing

In [4]:
print(string.punctuation)
print(stopwords.words("english")[100:110])

snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [5]:
# Your code

def remove_inline_js_css(text):
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL) # Regex to remove <script>...</script> tags and their content
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL) # Regex to remove <style>...</style> tags and their content
    return text

def remove_html_comments(text):
    return re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL) # Regex to match HTML comments

# Remove inline_JS_CSS
X_train = data['text'].apply(remove_inline_js_css)
X_test =  data['text'].apply(remove_inline_js_css)

# Apply the function to the 'text' column to remove HTML comments
X_train = X_train.apply(remove_html_comments)
X_test =  X_test.apply(remove_html_comments)

X_train.head()
X_test.head()

0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                             Will do.
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
Name: text, dtype: object

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [6]:
# Your code
def remove_others(text):
    text = re.sub(r'[^A-Za-z\s]+', '', text, flags=re.DOTALL) # Regex to remove all the special characters and numbers
    text = re.sub(r'\b\w\b', '', text, flags=re.DOTALL) # Regex to remove all single characters
    text = re.sub(r' {2,}', ' ', text, flags=re.DOTALL) # Regex to substitute multiple spaces with single space
    text = re.sub(r'\bb', '', text, flags=re.DOTALL) # Regex to remove prefixed 'b'
    text = re.sub(r'\s\s+', ' ', text, flags=re.DOTALL) # Substitute multiple spaces with single space

    return text.lower() # Convert to Lowercase

# Remove inline_JS_CSS
X_train = X_train.apply(remove_others)
X_test =  X_test.apply(remove_others)

X_train.head()
X_test.head()

0    dear sir strictly private business proposal am...
1                                              will do
2    noracheryl has emailed dozens of memos about h...
3    dear sirfmadamc know that this proposal might ...
4                                                  fyi
Name: text, dtype: object

## Now let's work on removing stopwords
Remove the stopwords.

In [7]:
# Your code
def removal(text): # Alex Code
    stop_words = set(stopwords.words('english')) # stopwords

    words = text.split()  # Split for tokenization
    words = [word for word in words if word not in stop_words] # Process everything except the stopwords

    return ' '.join(words)

# Remove stopwords
X_train = X_train.apply(removal)
X_test =  X_test.apply(removal)

X_train.head()
X_test.head()

0    dear sir strictly private business proposal mi...
1                                                     
2    noracheryl emailed dozens memos haiti weekend ...
3    dear sirfmadamc know proposal might e surprise...
4                                                  fyi
Name: text, dtype: object

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [8]:
# Your code
def lemmatization(text): # Alex Code
    lemmatizer = WordNetLemmatizer()

    words = text.split()  # Split for tokenization
    words = [lemmatizer.lemmatize(word) for word in words] # Lemmatize

    return ' '.join(words)

# Lemmatization
X_train = X_train.apply(lemmatization)
X_test =  X_test.apply(lemmatization)

X_train.head()
X_test.head()

0    dear sir strictly private business proposal mi...
1                                                     
2    noracheryl emailed dozen memo haiti weekend pl...
3    dear sirfmadamc know proposal might e surprise...
4                                                  fyi
Name: text, dtype: object

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [9]:
# Your code
# ham 0, spam = 1

all_words = ' '.join(X_train)

# Tokenize words
word_list = all_words.split()
print(word_list)

word_counts = Counter(word_list)
print(word_counts)

print("\nTo 10 words:")
for i in word_counts.most_common(10):
    print(i)

['dear', 'sir', 'strictly', 'private', 'business', 'proposal', 'mike', 'chukwu', 'manager', 'bill', 'exchange', 'foreign', 'remittance', 'department', 'zenith', 'international', 'bank', 'plc', 'writing', 'letter', 'ask', 'support', 'cooperation', 'carry', 'business', 'opportunity', 'department', 'discovered', 'abandoned', 'sum', 'fifteen', 'million', 'united', 'state', 'dollar', 'account', 'belongs', 'one', 'foreign', 'customer', 'died', 'along', 'entire', 'family', 'wife', 'two', 'child', 'november', 'plane', 'crash', 'since', 'heard', 'death', 'expecting', 'nextofkin', 'come', 'put', 'claim', 'money', 'heirbecause', 'cannot', 'release', 'fund', 'account', 'unless', 'someone', 'applies', 'claim', 'nextofkin', 'deceased', 'indicated', 'banking', 'guideline', 'unfortunately', 'neither', 'family', 'member', 'distant', 'relative', 'ever', 'appeared', 'claim', 'said', 'fund', 'upon', 'discoveryi', 'official', 'department', 'agreed', 'make', 'business', 'release', 'total', 'amount', 'accoun

## Extra features

In [10]:

# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data['money_mark'] = X_train.str.contains(money_simbol_list) * 1
data['suspicious_words'] = X_train.str.contains(suspicious_words) * 1
data['text_len'] = X_train.apply(lambda x: len(x))

'''
data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 
'''

data


,text,label,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,1,1,1504
1,Will do.,0,1,0,0
2,Nora--Cheryl has emailed dozens of memos about...,0,1,0,110
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,1,1,1391
4,fyi,0,1,0,3
...,...,...,...,...,...
995,So what's the latest? It sounds contradictory ...,0,1,0,61
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1,1,1,1879
997,Barb I will call to explain. Are you back in t...,0,1,0,29
998,Yang on travelNot free tonite.May work tomorrow,0,1,1,43


## How would work the Bag of Words with Count Vectorizer concept?

In [37]:
# Your code

# Create a Vectorizer Object
vectorizer = CountVectorizer()
 
vectorizer.fit(X_train)
 
# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)

# Encode the Document
vector = vectorizer.transform(X_train)
 
# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())

Vocabulary:  {'dear': 4287, 'sir': 16118, 'strictly': 16711, 'private': 13906, 'business': 2484, 'proposal': 14085, 'mike': 11024, 'chukwu': 2968, 'manager': 10603, 'bill': 2201, 'exchange': 6190, 'foreign': 6891, 'remittance': 14820, 'department': 4496, 'zenith': 20092, 'international': 9082, 'bank': 2012, 'plc': 13492, 'writing': 19758, 'letter': 10101, 'ask': 1619, 'support': 16913, 'cooperation': 3815, 'carry': 2670, 'opportunity': 12709, 'discovered': 4854, 'abandoned': 19, 'sum': 16876, 'fifteen': 6638, 'million': 11042, 'united': 18643, 'state': 16591, 'dollar': 5062, 'account': 177, 'belongs': 2128, 'one': 12604, 'customer': 4119, 'died': 4744, 'along': 830, 'entire': 5915, 'family': 6444, 'wife': 19500, 'two': 18453, 'child': 2918, 'november': 12140, 'plane': 13464, 'crash': 4025, 'since': 16059, 'heard': 7892, 'death': 4301, 'expecting': 6249, 'nextofkin': 11913, 'come': 3261, 'put': 14223, 'claim': 3020, 'money': 11240, 'heirbecause': 7934, 'cannot': 2590, 'release': 14758, 

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [38]:
# Your code
tf_idf_vectorizer = TfidfVectorizer() # TfidfVectorizer(ngram_range = (1, 2), max_features = 10000)

X_train_tf_idf = tf_idf_vectorizer.fit_transform(X_train)

X_test_tf_idf = tf_idf_vectorizer.transform(X_test)

## And the Train a Classifier?

In [42]:
# Your code

# Classifier will make prediction on my data.

# Train the classifier (MultinomialNB)
model = MultinomialNB()
model.fit(X_train_tf_idf, y)

# Make predictions on the validation set
y_pred = model.predict(X_test_tf_idf)

print("Predictions:", y_pred)

Predictions: [1 0 0 1 0 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 1 0 1 1
 1 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0 1 0 1
 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 1 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 0 1 1 0 1
 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0 0 0 1 0
 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 0 0 1 1 0 0 1 1 1 1 0 0 1 1 1 0
 0 0 0 1 1 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0
 1 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 1 1
 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 1 1 1 0 1 1 0 0 0 0 1
 1 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0
 1 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0
 1 0 0 1 1 0

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code